In [1]:
print("Bonjour")

Bonjour


### Importing Libraries

In [2]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np

### Taking Pictures Every 5s

In [3]:
import cv2
import time

cam = cv2.VideoCapture(0)

# Warm up the camera
time.sleep(2)

c = 0

while True:
    ret, frame = cam.read()

    if not ret:
        print("Failed to capture image.")
        break

    c += 1
    filename = f"data/captured_image_{c}.png"

    cv2.imshow("Captured", frame)
    cv2.imwrite(filename, frame)
    print(f"Saved {filename}")

    # Wait 5 seconds, exit if 'q' is pressed
    if cv2.waitKey(5000) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cam.release()


Saved data/captured_image_1.png


### Saves Images on click

In [5]:
import cv2

cam = cv2.VideoCapture(0)

if not cam.isOpened():
    print("Error: Camera not opened")
    exit()

cv2.namedWindow("Camera")

counter = 0
current_frame = None

def mouse_click(event, x, y, flags, param):
    global counter, current_frame
    if event == cv2.EVENT_LBUTTONDOWN:
        counter += 1
        filename = f"captured_{counter}.png"
        cv2.imwrite(filename, current_frame)
        print(f"Saved {filename}")

cv2.setMouseCallback("Camera", mouse_click)

while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame")
        break

    current_frame = frame.copy()
    cv2.imshow("Camera", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()


Saved captured_1.png
Saved captured_2.png


In [8]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
drawing_utils = mp.solutions.drawing_utils
drawing_styles = mp.solutions.drawing_styles

def run_filter_with_mediapipe_model(mediapipe_model, mediapipe_based_filter):
    """Run a media pipe model on each video frame grabbed by the webcam and draw results on it

    Args:
        mediapipe_model (): A mediapipe model
        mediapipe_based_filter (): a function to draw model results on frame

    Returns:
        np.ndarray, mediapipe model result
    """
    cap = cv2.VideoCapture(0)
    cv2.namedWindow("Face Swap",cv2.WINDOW_NORMAL)
    try:
        with mediapipe_model as model:
            while cap.isOpened():
                success, image = cap.read()

                if not success:
                    print("Ignoring empty camera frame.")
                    continue     # If loading a video, use 'break' instead of 'continue'.

                # Flip the image horizontally for a later selfie-view display, and convert
                # the BGR image to RGB.
                image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

                try:
                    results = model.process(image)
                except Exception:
                    results = None

                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                if results and results.pose_landmarks:
                    result_image = mediapipe_based_filter(image, results)
                else:
                    result_image = image

                cv2.imshow('Face Swap', result_image)

                if cv2.waitKey(5) & 0xFF == ord('q'):
                    break
                    
    finally:
        cap.release()
        cv2.destroyAllWindows()
    
    return image, results

In [14]:
# see https://google.github.io/mediapipe/solutions/holistic#python-solution-api
Holistic = mp.solutions.holistic.Holistic

def draw_holistic_results(image, results, show_hands=True, show_face=False, show_pose=True):
    if show_hands:
        drawing_utils.draw_landmarks(
            image,
            results.left_hand_landmarks,
            mp.solutions.holistic.HAND_CONNECTIONS,
            connection_drawing_spec=drawing_styles.get_default_hand_connections_style()
        )

        drawing_utils.draw_landmarks(
            image,
            results.right_hand_landmarks,
            mp.solutions.holistic.HAND_CONNECTIONS,
            connection_drawing_spec=drawing_styles.get_default_hand_connections_style()
        )

    if show_face:
        drawing_utils.draw_landmarks(
            image,
            results.face_landmarks,
            mp.solutions.holistic.FACEMESH_CONTOURS,
            landmark_drawing_spec=drawing_utils.DrawingSpec(thickness=0, circle_radius=0, color=(255, 255, 255)),
            connection_drawing_spec=drawing_styles.get_default_face_mesh_contours_style()
        )

    if show_pose:
        drawing_utils.draw_landmarks(
            image,
            results.pose_landmarks,
            mp.solutions.holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=drawing_styles.get_default_pose_landmarks_style()
        )
    return image

In [15]:
last_image, last_results = run_filter_with_mediapipe_model(
    mediapipe_model=Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5),
    mediapipe_based_filter=draw_holistic_results
)